In [52]:
import pandas as pd
import re

chatbot_data = pd.read_csv('./korean_chatbot_data/ChatbotData.csv')
chatbot_data = chatbot_data.sample(frac=1).reset_index(drop=True)

print(chatbot_data[0: 10])


                       Q                           A  label
0                실수한거 같아                   잘 생각해보세요.      0
1            커피 좀 줄여야겠어.                  과해도 안 좋아요.      0
2                모른척 해줬어        그게 서로에게 좋았던 선택일 거예요.      0
3                   답답해서                 좋은 생각만 하세요.      1
4             돈이 천원밖에 없어           돈 없어도 할 수 있는게 많아요      0
5     짝사랑만큼 고통스러운 건 없겠지.    짝사랑 만큼 감정소모가 큰 건 없을 거예요.      2
6  3달이지났는데 이제야 헤어짐을 확신했네              이제야 실감이 나나 봐요.      1
7                    답답해  가까운 곳으로 여행을 가보는 것도 좋을 거예요.      1
8                힘드네 장거리        물리적인 거리를 무시하지 못하니까요.      1
9              택배 왜 안오지?              송장 번호를 확인해보세요.      0


In [53]:
EMBEDDING_DIM = 256
UNITS = 1024
MAX_LEN = 30
TIME_STEPS = MAX_LEN


In [54]:
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Attention
from tensorflow.keras import Model

okt = Okt()

def clean_sentence (sentence):
  sentence = re.sub(r'[^0-9ㄱ-ㅎㅏ-ㅣ가-힣 ]', r'', sentence)
  return sentence

def process_morph (sentence):
  return ' '.join(okt.morphs(sentence))

def clean_and_morph(sentence):
  sentence = clean_sentence(sentence)
  sentence = process_morph(sentence)
  return sentence

def attach_answer_token (sentence):
  return ('<START> ' + sentence, sentence + ' <END>')

def preprocess (questions, answers):
  ret_questions = []
  ret_answer_ins = []
  ret_answer_outs = []

  for question in questions:
    question_processed = process_morph(clean_sentence(question))
    ret_questions.append(question_processed)

  for answer in answers:
    answer_in, answer_out = attach_answer_token(process_morph(clean_sentence(answer)))
    ret_answer_ins.append(answer_in)
    ret_answer_outs.append(answer_out)
  
  return ret_questions, ret_answer_ins, ret_answer_outs

class TransformUtils:
  def __init__ (self, tokenizer, max_len, start_token, end_token):
    self.tokenizer = tokenizer
    self.max_len = max_len
    self.start_token = start_token
    self.end_token = end_token
  
  def convert_sentences_to_vectors (self, sentences):
    ret = []
    for i in range(len(sentences)):
      sentence = clean_sentence(sentences[i])
      sentence = process_morph(sentences[i])
      ret.append(sentence)
    ret = self.tokenizer.texts_to_sequences(sentences)
    ret = pad_sequences(ret, maxlen=self.max_len, truncating='post', padding='post')
    ret = tf.convert_to_tensor(ret, dtype=tf.int32)
    return ret

  def convert_vectors_to_sentences (self, vectors):
    ret = []
    for vector in vectors:
      sentence = ''
      for vi in vector:
        if vi <= 0 or self.tokenizer.index_word[vi] is None:
          sentence += '<None>'
        else:
          sentence += self.tokenizer.index_word[vi]
        sentence += ' '
      ret.append(sentence)
    return ret


In [55]:
questions, answer_ins, answer_outs = preprocess(chatbot_data['Q'], chatbot_data['A'])
print(questions[0: 10], answer_ins[0: 10], answer_outs[0: 10])
tokenizer = Tokenizer(filters='', lower=False, oov_token='<OOV>')
all_sentences = questions + answer_ins + answer_outs
tokenizer.fit_on_texts(all_sentences)

START_TOKEN = tokenizer.word_index['<START>']
END_TOKEN = tokenizer.word_index['<END>']
VOCAB_SIZE = len(tokenizer.word_index) + 1

['실수 한 거 같아', '커피 좀 줄여야겠어', '모른 척 해줬어', '답답해서', '돈 이 천원 밖에 없어', '짝사랑 만큼 고통스러운 건 없겠지', '3 달이 지났는데 이제야 헤어짐을 확신 했네', '답답해', '힘드네 장거리', '택배 왜 안 오지'] ['<START> 잘 생각 해보세요', '<START> 과 해도 안 좋아요', '<START> 그게 서로 에게 좋았던 선택 일 거 예요', '<START> 좋은 생각 만 하세요', '<START> 돈 없어도 할 수 있는게 많아요', '<START> 짝사랑 만큼 감정 소모 가 큰 건 없을 거 예요', '<START> 이제야 실감 이 나나 봐요', '<START> 가까운 곳 으로 여행 을 가보는 것 도 좋을 거 예요', '<START> 물리 적 인 거리 를 무시 하지 못 하니까 요', '<START> 송장 번호 를 확인 해보세요'] ['잘 생각 해보세요 <END>', '과 해도 안 좋아요 <END>', '그게 서로 에게 좋았던 선택 일 거 예요 <END>', '좋은 생각 만 하세요 <END>', '돈 없어도 할 수 있는게 많아요 <END>', '짝사랑 만큼 감정 소모 가 큰 건 없을 거 예요 <END>', '이제야 실감 이 나나 봐요 <END>', '가까운 곳 으로 여행 을 가보는 것 도 좋을 거 예요 <END>', '물리 적 인 거리 를 무시 하지 못 하니까 요 <END>', '송장 번호 를 확인 해보세요 <END>']


In [56]:
transform_utils = TransformUtils(tokenizer, MAX_LEN, START_TOKEN, END_TOKEN)

In [57]:
class Encoder (Model):
  def __init__ (self, input_vocab_size, embedding_dim, enc_units):
    super(Encoder, self).__init__()
    self.input_vocab_size = input_vocab_size
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(self.input_vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')

  def call (self, inputs, state=None):
    vectors = self.embedding(inputs)
    output, state = self.gru(vectors, initial_state=state)

    return output, state

In [58]:
class Decoder (Model):
  def __init__ (self, output_vocab_size, embedding_dim, dec_units):
    super(Decoder, self).__init__()
    self.output_vocab_size = output_vocab_size
    self.embedding_dim = embedding_dim
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(self.output_vocab_size, self.embedding_dim)
    self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences=True, return_state=True, recurrent_initializer='glorot_uniform')
    self.attention = Attention()
    self.fc = tf.keras.layers.Dense(self.output_vocab_size, activation='softmax')

  def call (self, inputs, enc_outputs, state=None):
    vectors = self.embedding(inputs)
    rnn_output, rnn_output_state = self.gru(vectors, initial_state=state)
    context_vector = self.attention(inputs=[rnn_output, enc_outputs])
    context_and_output = tf.concat([rnn_output, context_vector], axis=-1)
    attention_vector = self.fc(context_and_output)
    return attention_vector, rnn_output_state


In [59]:
class Seq2SeqTrainer (Model):
  def __init__ (self, vocab_size, embedding_dim, units):
    super(Seq2SeqTrainer, self).__init__()
    self.encoder = Encoder(vocab_size, embedding_dim, units)
    self.decoder = Decoder(vocab_size, embedding_dim, units)

  def __init__ (self, encoder, decoder):
    super(Seq2SeqTrainer, self).__init__()
    self.encoder = encoder
    self.decoder = decoder

  def call (self, inputs):
    questions, answer_ins = inputs
    enc_outputs, enc_state = self.encoder(questions)
    dec_result, dec_state = self.decoder(answer_ins, enc_outputs, state=enc_state)
    return dec_result


In [60]:
class TrainLoss (tf.keras.losses.Loss):
  def __init__ (self):
    super(TrainLoss, self).__init__()
    self.loss = tf.keras.losses.SparseCategoricalCrossentropy()
  
  def call (self, y_t, y_pred):
    loss = self.loss(y_t, y_pred)
    return tf.reduce_sum(loss)


In [61]:
class Translator:
  def __init__ (self, encoder, decoder, transform_utils, start_token, end_token):
    self.encoder = encoder
    self.decoder = decoder
    self.transform_utils = transform_utils
    self.start_token = start_token
    self.end_token = end_token
  
  def translate (self, inputs):
    return transform_utils.convert_vectors_to_sentences(self.get_translated_vectors(inputs))
  
  def get_translated_vectors (self, inputs):
    ret = []
    vectors = transform_utils.convert_sentences_to_vectors(inputs)
    for vector in vectors:
      ret_vector = [self.start_token]
      enc_output, enc_state = self.encoder(tf.convert_to_tensor([vector], dtype=tf.int32))
      dec_state = enc_state
      cnt = 0
      while ret_vector[-1] != self.end_token and cnt < 30:
        dec_result, dec_state = self.decoder(tf.convert_to_tensor([[ret_vector[-1]]], dtype=tf.int32), enc_output, state=dec_state)
        ret_vector.append(tf.math.argmax(dec_result[0][0]).numpy())
        cnt += 1
      ret.append(ret_vector)
      
    return ret


In [62]:
BATCH_SIZE = 64

question_vectors, answer_in_vectors, answer_out_vectors = \
  transform_utils.convert_sentences_to_vectors(questions), \
  transform_utils.convert_sentences_to_vectors(answer_ins), \
  transform_utils.convert_sentences_to_vectors(answer_outs)

encoder = Encoder(VOCAB_SIZE, EMBEDDING_DIM, UNITS)
encoder(Input(shape=(TIME_STEPS, ), dtype=tf.int32))
encoder.summary()
decoder = Decoder(VOCAB_SIZE, EMBEDDING_DIM, UNITS)
decoder(Input(shape=(TIME_STEPS, ), dtype=tf.int32), Input(shape=(TIME_STEPS, UNITS, ), dtype=tf.float32), state=Input(shape=(UNITS, ), dtype=tf.float32))
decoder.summary()
trainer = Seq2SeqTrainer(encoder, decoder)
trainer([Input(shape=(TIME_STEPS, ), dtype=tf.int32), Input(shape=(TIME_STEPS, ), dtype=tf.int32)])
trainer.summary()
trainer.compile(optimizer=tf.optimizers.Adam(learning_rate=0.001), loss=TrainLoss(), metrics=['acc'])

with tf.device('/device:GPU:0'):
  history = trainer.fit([question_vectors, answer_in_vectors], answer_out_vectors, batch_size=BATCH_SIZE, epochs=20, validation_split=0)
print(history.history)

trainer.save_weights('./model/trainer')

encoder.save_weights('./model/translator/encoder')
decoder.save_weights('./model/translator/decoder')


Model: "encoder_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    multiple                  3235328   
                                                                 
 gru_10 (GRU)                multiple                  3938304   
                                                                 
Total params: 7,173,632
Trainable params: 7,173,632
Non-trainable params: 0
_________________________________________________________________
Model: "decoder_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    multiple                  3235328   
                                                                 
 gru_11 (GRU)                multiple                  3938304   
                                                                 
 attention_5 (Attention)    

In [63]:
encoder = Encoder(VOCAB_SIZE, EMBEDDING_DIM, UNITS)
encoder.load_weights('./model/translator/encoder')
encoder(Input(shape=(TIME_STEPS, ), dtype=tf.int32))
encoder.summary()
decoder = Decoder(VOCAB_SIZE, EMBEDDING_DIM, UNITS)
decoder.load_weights('./model/translator/decoder')
decoder(Input(shape=(TIME_STEPS, ), dtype=tf.int32), Input(shape=(TIME_STEPS, UNITS, ), dtype=tf.float32), state=Input(shape=(UNITS, ), dtype=tf.float32))
decoder.summary()

translator = Translator(encoder, decoder, transform_utils, START_TOKEN, END_TOKEN)


Model: "encoder_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    multiple                  3235328   
                                                                 
 gru_12 (GRU)                multiple                  3938304   
                                                                 
Total params: 7,173,632
Trainable params: 7,173,632
Non-trainable params: 0
_________________________________________________________________
Model: "decoder_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_13 (Embedding)    multiple                  3235328   
                                                                 
 gru_13 (GRU)                multiple                  3938304   
                                                                 
 attention_6 (Attention)    

In [73]:
sentences = [
  '뭐 할까',
  '커피 마시고 싶다',
  '인 성 문제 있는거 아니야',
  '공부 하기 싫다',
  '숙제 가 너무 많다',
  '먹을 꺼 추천 좀',
  '재미 난 거 없나',
  '주식 공부 할 까',
  '가스 불 켜고 나갔어',
  '우리 집 에 불 났어',
  '여행 가고 싶습니다',
  '축구 하고 싶다',
  '술 중독 인거 같아',
  '나 이상한가',
  '나 잘 살 수 있겠지',
  '집에 가고 싶다',
  '배 타고 싶다'
]
# print('\n'.join(transform_utils.convert_vectors_to_sentences(
#   transform_utils.convert_sentences_to_vectors(sentences).numpy()
# )))
# print('\n')
# print('\n'.join(translator.translate(sentences)))
for sentence in sentences:
  print(sentence + ':', end=' ')
  print(''.join(translator.translate([sentence])))


뭐 할까: <START> 저 랑 놀아요 <END> 
커피 마시고 싶다: <START> 저 도 커피 좋아해요 <END> 
인 성 문제 있는거 아니야: <START> 새로운 무언가 를 해보는건 좋겠어요 <END> 
공부 하기 싫다: <START> 같이 수다 떨면서 놀까 요 <END> 
숙제 가 너무 많다: <START> 미리 미리 해야죠 <END> 
먹을 꺼 추천 좀: <START> 냉장고 파먹기 해보세요 <END> 
재미 난 거 없나: <START> 한번 더 울어 보세요 <END> 
주식 공부 할 까: <START> 남 에게 피 할 수 있을 거 예요 <END> 
가스 불 켜고 나갔어: <START> 빨리 집 에 돌아가서 끄고 나오세요 <END> 
우리 집 에 불 났어: <START> 집 에 도움 이 되려고 노력 해보세요 <END> 
여행 가고 싶습니다: <START> 좋은 여행 되길 바랍니다 <END> 
축구 하고 싶다: <START> 잘 될 거 예요 <END> 
술 중독 인거 같아: <START> 술 많이 드시면 더 무너져요 <END> 
나 이상한가: <START> 지극히 평범하면서 지극히 특별하죠 <END> 
나 잘 살 수 있겠지: <START> 잘 살 수 있을 거 예요 <END> 
집에 가고 싶다: <START> 미리 충전 하세요 <END> 
배 타고 싶다: <START> 어찌 하면 좋을까요 <END> 
